In [125]:
#instantiate cursor and load libraries
import json
import pandas as pd
import requests
import mysql.connector
import config_copy
import time
import re
import requests
import urllib.request
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup as BS

## Call NYT API to get ISBNs of NYT Bestsellers

In [11]:
# create list to hold all data
all_best_sellers = []

for i in range(0, 32561, 20):
    params = {'api-key': config.api_key,
                     'offset': i}
    url = 'https://api.nytimes.com/svc/books/v3/lists/best-sellers/history.json'
    response = requests.get(url, params = params)
    data = response.json()

    for book in data['results']:
        
        each_book = {}
        
        try:
            each_book['title'] = book['title']
            each_book['author'] = book['author']
            each_book['publisher'] = book['publisher']
            each_book['ISBN'] = book['isbns']
            each_book['publish_date'] = book['ranks_history'][0]['published_date']

            # add the dictionary to master list
            all_best_sellers.append(each_book)

        except IndexError:
            continue

    # print what page we are on for auditing purposes
    print(i)

    # wait so we don't hit the API's per minute call limit
    time.sleep(6)


29200
29220
29240
29260
29280
29300
29320
29340
29360
29380
29400
29420
29440
29460
29480
29500
29520
29540
29560
29580
29600
29620
29640
29660
29680
29700
29720
29740
29760
29780
29800
29820
29840
29860
29880
29900
29920
29940
29960
29980
30000
30020
30040
30060
30080
30100
30120
30140
30160
30180
30200
30220
30240
30260
30280
30300
30320
30340
30360
30380
30400
30420
30440
30460
30480
30500
30520
30540
30560
30580
30600
30620
30640
30660
30680
30700
30720
30740
30760
30780
30800
30820
30840
30860
30880
30900
30920
30940
30960
30980
31000
31020
31040
31060
31080
31100
31120
31140
31160
31180
31200
31220
31240
31260
31280
31300
31320
31340
31360
31380
31400
31420
31440
31460
31480
31500
31520
31540
31560
31580
31600
31620
31640
31660
31680
31700
31720
31740
31760
31780
31800
31820
31840
31860
31880
31900
31920
31940
31960
31980
32000
32020
32040
32060
32080
32100
32120
32140
32160
32180
32200
32220
32240
32260
32280
32300
32320
32340
32360
32380
32400
32420
32440
32460
32480
32500
3252

#### Load data in and reformat

In [3]:
bestsellers_df= pd.read_csv('bestsellers.csv', index_col=0)
bestsellers_df.head()

,title,author,publisher,ISBN,publish_date
0,"""I GIVE YOU MY BODY ...""",Diana Gabaldon,Dell,"[{'isbn10': '0399178570', 'isbn13': '978039917...",2016-09-04
1,"""MOST BLESSED OF THE PATRIARCHS""",Annette Gordon-Reed and Peter S Onuf,Liveright,"[{'isbn10': '0871404427', 'isbn13': '978087140...",2016-05-01
2,#ASKGARYVEE,Gary Vaynerchuk,HarperCollins,"[{'isbn10': '0062273124', 'isbn13': '978006227...",2016-04-10
3,#GIRLBOSS,Sophia Amoruso,Portfolio/Penguin/Putnam,"[{'isbn10': '039916927X', 'isbn13': '978039916...",2016-03-13
4,#IMOMSOHARD,Kristin Hensley and Jen Smedley,HarperOne,"[{'isbn10': '006285769X', 'isbn13': '978006285...",2019-04-21


In [4]:
def get_isbn10(x):
    try:
        split = x.split()[1]
        return str(re.findall('(\d+X?)', split)[0])
    except:
        return None

In [5]:
bestsellers_df['ISBN10'] = bestsellers_df['ISBN'].apply(lambda x: get_isbn10(x))

In [6]:
bestsellers_df

,title,author,publisher,ISBN,publish_date,ISBN10
0,"""I GIVE YOU MY BODY ...""",Diana Gabaldon,Dell,"[{'isbn10': '0399178570', 'isbn13': '978039917...",2016-09-04,0399178570
1,"""MOST BLESSED OF THE PATRIARCHS""",Annette Gordon-Reed and Peter S Onuf,Liveright,"[{'isbn10': '0871404427', 'isbn13': '978087140...",2016-05-01,0871404427
2,#ASKGARYVEE,Gary Vaynerchuk,HarperCollins,"[{'isbn10': '0062273124', 'isbn13': '978006227...",2016-04-10,0062273124
3,#GIRLBOSS,Sophia Amoruso,Portfolio/Penguin/Putnam,"[{'isbn10': '039916927X', 'isbn13': '978039916...",2016-03-13,039916927X
4,#IMOMSOHARD,Kristin Hensley and Jen Smedley,HarperOne,"[{'isbn10': '006285769X', 'isbn13': '978006285...",2019-04-21,006285769X
...,...,...,...,...,...,...
7788,ZOO,James Patterson and Michael Ledwidge,Vision,"[{'isbn10': '0316097446', 'isbn13': '978031609...",2015-08-30,0316097446
7789,ZOO 2,James Patterson with Max DiLallo,"Little, Brown","[{'isbn10': '0316317128', 'isbn13': '978031631...",2016-07-10,0316317128
7790,ZUCKED,Roger McNamee,Penguin Press,"[{'isbn10': '0525561358', 'isbn13': '978052556...",2019-03-17,0525561358
7791,_____ MY DAD SAYS,Justin Halpern,HarperCollins,"[{'isbn10': '0061992704', 'isbn13': '978006199...",2016-07-10,0061992704


In [553]:
# for book in list_of_NYT_books:
#     for row in bestsellers_df.itertuples():
#         if book['title'] == getattr(row, 'title'):
#             book['ISBN'] = getattr(row, 'ISBN10')
#             print('updated')


updated
updated
updated
updated
updated
updated


In [7]:
years = [2017, 2018, 2019]
ISBNs = []

for row in bestsellers_df.itertuples():
    if int(getattr(row, 'publish_date').split('-')[0]) in years:
        try:
            ISBN = getattr(row, 'ISBN10')
            
            ISBNs.append(ISBN)
            
        except:
            continue

In [125]:
# pd.DataFrame(all_best_sellers).to_csv('bestsellers.csv')

# Attempt to scrape NYT Bestellers from Goodreads using ISBNs

In [8]:
# this function takes in a list of ISBNs that were gathered from the NYT API 
# and returns a list of dictionaries, where each dictionary reflects data 
# about the NYT bestseller

# list_of_NYT_books = []

def get_NYTbook_info(ISBNs):
    """
    get_NYTbook_info():
    Function returns a list of dictionaries with features describing each NYT bestseller
    Params:
        ISBNs: list of ISBNs for each NYT bestseller
    Returns:
        List of dictionaries describing bestselling books
    """
    each_book = {}

    for ISBN in ISBNs[400:]:
        
#         for book in list_of_NYT_books:
# #             print(book)
#             if ISBN == book['ISBN'][3:] or ISBN == book['ISBN']:
#                 print(ISBN)
#                 print('copy')
                
#             else:
        try:
            each_book = {}

            #set up chrome driver through selenium
            driver = webdriver.Chrome(executable_path = '/Applications/chromedriver')

            # log into goodreads website 
            driver.get('https://www.goodreads.com/')

            # wait for website to load
    #                 time.sleep(2)

            # get the webpage for each ISBN 
            driver.find_element_by_name('query').send_keys(ISBN,Keys.ENTER)
            time.sleep(2) 

            #get URL for webpage
            url = driver.current_url
            type(url)

            #scrape
            page = BS(requests.get(str(url)).content, 'html.parser')

            each_book['title'] = get_title(page)
            each_book['author'] = get_author(page)
            each_book['rating'] = get_rating(page)
            each_book['pages'] = get_pages(page)
            each_book['publisher'] = get_publisher(page)
            each_book['ISBN'] = ISBN
            each_book['genre'] = get_genre(page)
            each_book['series'] = get_series(page)
            each_book['author_followers'] = get_followers(page)
            each_book['publish_date'] = get_release_date(page)

            list_of_NYT_books.append(each_book)

            print(f"Inserted {each_book['title']}")
        
        except:
            continue

    time.sleep(2)
    
    return list_of_NYT_books
        

In [128]:
len(list_of_NYT_books)

880

In [131]:
# list_of_NYT_books = []
years = [2019]
genres = ['debut-novel','young-adult-fiction-books', 'young-adult-fantasy-books', 'childrens-books', 'picture-books',
    'fiction-books', 'mystery-thriller-books', 'historical-fiction-books', 'fantasy-books', 
          'romance-books', 'science-fiction-books', 'horror-books', 'humor-books', 'nonfiction-books',
          'memoir-autobiography-books', 'history-biography-books', 'science-technology-books',
          'food-cookbooks', 'graphic-novels-comics', 'poetry-books']
          

def get_2019books():
    
    for year in years:
        
        print(year)
        
        for genre in genres:
            
            print(genre)
            
            each_book = {}
            
            url = f'https://www.goodreads.com/choiceawards/best-{genre}-{year}'
            resp = urllib.request.urlopen(url)
            soup = BS(resp, from_encoding=resp.info().get_param('charset'))
            
            for info in soup.find_all(href=True):
#                 print(info)
                URLs = []
                if 'from_choice=true' in info['href']:
                    end_url = info['href']
                    URLs.append(end_url)
                
                for URL in URLs:
                    page=f'http://goodreads.com{end_url}'
                    print(page)

                    resp = urllib.request.urlopen(page)
                    content = BS(resp, 'html.parser')

                    each_book['title'] = get_title(content)
                    each_book['author'] = get_author(content)
                    each_book['rating'] = get_rating(content)
                    each_book['pages'] = get_pages(content)
                    each_book['publisher'] = get_publisher(content)
                    each_book['ISBN'] = get_ISBN(content)
                    each_book['genre'] = get_genre(content)
                    each_book['series'] = get_series(content)
                    each_book['author_followers'] = get_followers(content)
                    each_book['publish_date'] = get_release_date(content)

                    list_of_NYT_books.append(each_book)

                    print(f"Inserted {each_book['title']}")

                    continue
                          
    pd.DataFrame(list_of_NYT_books).to_csv('all_books.csv')

    return list_of_NYT_books


In [144]:
# list_of_NYT_books = []
years = [2015, 2016]
genres = [
#     'mystery-thriller-books', 'historical-fiction-books', 'fantasy-books', 
          'romance-books', 'science-fiction-books', 'horror-books', 'humor-books', 'nonfiction-books',
          'memoir-autobiography-books', 'history-biography-books', 'science-technology-books',
          'food-cookbooks', 'graphic-novels-comics', 'poetry-books', 'debut-goodreads-author',
          'young-adult-fiction-books', 'young-adult-fantasy-books', 'childrens-books', 'picture-books']

def get_2015_2016_books():
    
    for year in years:
        
        print(year)
        
        for genre in genres:
            
            print(genre)
            
            each_book = {}
            
            url = f'https://www.goodreads.com/choiceawards/best-{genre}-{year}'
            resp = urllib.request.urlopen(url)
            soup = BS(resp, from_encoding=resp.info().get_param('charset'))
            
            for info in soup.find_all(href=True):
#                 print(info)
                URLs = []
                if 'from_choice=true' in info['href']:
                    end_url = info['href']
                    URLs.append(end_url)
                
                for URL in URLs:
                    page=f'http://goodreads.com{end_url}'
                    print(page)

                    resp = urllib.request.urlopen(page)
                    content = BS(resp, 'html.parser')

                    each_book['title'] = get_title(content)
                    each_book['author'] = get_author(content)
                    each_book['rating'] = get_rating(content)
                    each_book['pages'] = get_pages(content)
                    each_book['publisher'] = get_publisher(content)
                    each_book['ISBN'] = get_ISBN(content)
                    each_book['genre'] = get_genre(content)
                    each_book['series'] = get_series(content)
                    each_book['author_followers'] = get_followers(content)
                    each_book['publish_date'] = get_release_date(content)

                    list_of_NYT_books.append(each_book)

                    print(f"Inserted {each_book['title']}")

                    continue

    return list_of_NYT_books
    

In [127]:
pd.DataFrame(list_of_NYT_books).to_csv('all_books.csv')

In [108]:
pd.DataFrame(list_of_NYT_books).to_csv('goodreads_books.csv')

In [290]:
pd.DataFrame(list_of_NYT_books).to_csv('first_383_bestsellers.csv')

In [63]:
def get_ISBN(page):
    try:
        return page.find_all(class_='infoBoxRowItem')[1].get_text(strip=True).split('(')[0]
    except:
        return None

In [9]:
def get_title(page):
    try:
        return page.find(id = 'bookTitle').get_text(strip=True)
    except:
        return None

In [10]:
def get_author(page):
    try:
        return page.find('span', itemprop = 'name').get_text(strip=True)
    except:
        return None

In [11]:
def get_rating(page):
    try:
        return float(page.find('span', itemprop = 'ratingValue').get_text(strip=True))
    except:
        return None

In [12]:
def get_pages(page):
    try:
        return page.find('span', itemprop = 'numberOfPages').get_text(strip=True).split(' ')[0]
    except:
        return None

In [13]:
def get_publisher(page):
    try:
        details = page.find_all('div', 'row')
        row = details[1].get_text(strip=True)
        publisher = row.split('by')[1].strip()
        return publisher
    except:
        return None

In [14]:
def get_genre(page):
    try:
        genre = page.find_all('div', class_='elementList')[0].get_text(strip=True)
        genre = re.findall('(\w+[a-zA-Z])', genre)[0]
        return genre
    except: 
        return None

In [15]:
import re
def solve(s):                                             
    return re.sub(r'(\d)(st|nd|rd|th)', r'\1', s)

In [16]:
import datetime

In [17]:
def get_release_date(page):
    try:
        details = page.find_all('div', 'row')
        row = details[1].get_text(strip=True)
        row1 = row.split('Published')[1].strip()
        date = row1.split('by')[0].strip()
        date = datetime.datetime.strptime(solve(date), "%B %d %Y")
        return date
    except:
        return None

In [214]:
def get_series(page):
    try:
        if page.find_all('div', class_='infoBoxRowTitle')[3].get_text() == 'Series':
            return 1
        else:
            return 0
    except:
        return 0

In [19]:
def get_followers(page):
    try:
        bio = page.find('div', class_ = 'bookAuthorProfile')
        parse = bio.get_text(strip=True)
        parse1 = re.findall('(\d+(?:[\.,]\d+)?)',parse)[0]
        final = parse1.replace(',', '')
        return int(final)
    except:
        return None

# Scrape 2018

In [222]:
books_2018 = []
years = [2018]
genres = ['debut-author', 'young-adult-fiction-books', 'young-adult-fantasy-books', 'childrens-books', 
          'picture-books','fiction-books', 'mystery-thriller-books', 'historical-fiction-books', 'fantasy-books', 
          'romance-books', 'science-fiction-books', 'horror-books', 'humor-books', 'nonfiction-books',
          'memoir-autobiography-books', 'history-biography-books', 'science-technology-books',
          'food-cookbooks', 'graphic-novels-comics', 'poetry-books']

def get_2018books():
    
    for year in years:
        
        print(year)
        
        for genre in genres:
            
            print(genre)
            
            url = f'https://www.goodreads.com/choiceawards/best-{genre}-{year}'
            resp = urllib.request.urlopen(url)
            soup = BS(resp, from_encoding=resp.info().get_param('charset'))
            
            for info in soup.find_all(href=True):
#                 print(info)

                if 'from_choice=true' in info['href']:
                    end_url = info['href']

                    each_book = {}

                    resp = urllib.request.urlopen(f'https://www.goodreads.com{end_url}')
                    content = BS(resp, 'html.parser')

                    each_book['title'] = get_title(content)
                    each_book['author'] = get_author(content)
                    each_book['rating'] = get_rating(content)
                    each_book['pages'] = get_pages(content)
                    each_book['publisher'] = get_publisher(content)
                    each_book['ISBN'] = get_ISBN(content)
                    each_book['genre'] = get_genre(content)
                    each_book['series'] = get_series(content)
                    each_book['author_followers'] = get_followers(content)
                    each_book['publish_date'] = get_release_date(content)

                    books_2018.append(each_book)

                    print(f"Inserted {each_book['title']}")

    pd.DataFrame(books_2018).to_csv('books_2018.csv')

    return books_2018
    

In [223]:
get_2018books()

2018
debut-author
Inserted Children of Blood and Bone
Inserted Children of Blood and Bone
Inserted Children of Blood and Bone
Inserted The Woman in the Window
Inserted The Tattooist of Auschwitz
Inserted The Kiss Quotient
Inserted To Kill a Kingdom
Inserted There There
Inserted The Poppy War
Inserted The Astonishing Color of After
Inserted Something in the Water
Inserted The Chalk Man
Inserted An Absolutely Remarkable Thing
Inserted A Place for Us
Inserted The Map of Salt and Stars
Inserted The 7½ Deaths of Evelyn Hardcastle
Inserted Sky in the Deep
Inserted Girls Burn Brighter
Inserted Everything Here Is Beautiful
Inserted Fruit of the Drunken Tree
Inserted Witchmark
Inserted The Terminal List
young-adult-fiction-books
Inserted Leah on the Offbeat
Inserted Leah on the Offbeat
Inserted Leah on the Offbeat
Inserted Sadie
Inserted What If It's Us
Inserted All the Little Lights
Inserted The Lady's Guide to Petticoats and Piracy
Inserted The Astonishing Color of After
Inserted Save the Dat

Inserted Your Dad Stole My Rake: And Other Family Dilemmas
Inserted Feminasty: The Complicated Woman's Guide to Surviving the Patriarchy Without Drinking Herself to Death
Inserted Maeve in America: Essays by a Girl from Somewhere Else
nonfiction-books
Inserted I'll Be Gone in the Dark
Inserted I'll Be Gone in the Dark
Inserted I'll Be Gone in the Dark
Inserted Girl, Wash Your Face: Stop Believing the Lies About Who You Are so You Can Become Who You Were Meant to Be
Inserted Fear: Trump in the White House
Inserted Whiskey in a Teacup: What Growing Up in the South Taught Me About Life, Love, & Baking Biscuits
Inserted The Library Book
Inserted 21 Lessons for the 21st Century
Inserted Not That Bad: Dispatches from Rape Culture
Inserted So You Want to Talk About Race
Inserted 12 Rules for Life: An Antidote to Chaos
Inserted Factfulness: Ten Reasons We're Wrong About the World – and Why Things Are Better Than You Think
Inserted Bad Blood: Secrets and Lies in a Silicon Valley Startup
Inserte

Inserted Be Prepared
Inserted Bingo Love
Inserted Gone Rogue
Inserted Check, Please! Book 1: # Hockey
Inserted Sabrina
Inserted Imperial Phase, Part II
Inserted Monstress, Vol. 3: Haven
Inserted Hey, Kiddo
Inserted Black Hammer, Vol. 2: The Event
Inserted Black Bolt, Vol. 1: Hard Time
poetry-books
Inserted The Witch Doesn't Burn in This One
Inserted The Witch Doesn't Burn in This One
Inserted The Witch Doesn't Burn in This One
Inserted The Poet X
Inserted Useless Magic: Lyrics and Poetry
Inserted The Dark Between Stars
Inserted Taking the Arrow Out of the Heart
Inserted Rebound
Inserted If They Come for Us
Inserted Take Me with You
Inserted Sea of Strangers
Inserted She Felt Like Feeling Nothing
Inserted Helium
Inserted For Every One
Inserted Wade in the Water
Inserted The Future
Inserted A Love Letter from the Girls Who Feel Everything
Inserted Heart Talk: Poetic Wisdom for a Better Life
Inserted American Sonnets for My Past and Future Assassin
Inserted Blood Water Paint
Inserted The 

[{'title': 'Children of Blood and Bone',
  'author': 'Tomi Adeyemi',
  'rating': 4.16,
  'pages': '544',
  'publisher': 'Henry Holt Books for Young Readers',
  'ISBN': '1250170974',
  'genre': 'Fantasy',
  'series': 1,
  'author_followers': 11314,
  'publish_date': datetime.datetime(2018, 3, 6, 0, 0)},
 {'title': 'Children of Blood and Bone',
  'author': 'Tomi Adeyemi',
  'rating': 4.16,
  'pages': '544',
  'publisher': 'Henry Holt Books for Young Readers',
  'ISBN': '1250170974',
  'genre': 'Fantasy',
  'series': 1,
  'author_followers': 11314,
  'publish_date': datetime.datetime(2018, 3, 6, 0, 0)},
 {'title': 'Children of Blood and Bone',
  'author': 'Tomi Adeyemi',
  'rating': 4.16,
  'pages': '544',
  'publisher': 'Henry Holt Books for Young Readers',
  'ISBN': '1250170974',
  'genre': 'Fantasy',
  'series': 1,
  'author_followers': 11314,
  'publish_date': datetime.datetime(2018, 3, 6, 0, 0)},
 {'title': 'The Woman in the Window',
  'author': 'A.J. Finn',
  'rating': 3.93,
  'page

In [228]:
# books_2015_2016 = []
years = [2015, 2016]
# genres = ['fiction-books','mystery-thriller-books', 'historical-fiction-books', 'fantasy-books', 
genres = ['romance-books', 'science-fiction-books', 'horror-books', 'humor-books', 'nonfiction-books',
          'memoir-autobiography-books', 'history-biography-books', 'science-technology-books',
          'food-cookbooks', 'graphic-novels-comics', 'poetry-books', 'debut-goodreads-author',
          'young-adult-fiction-books', 'young-adult-fantasy-books', 'childrens-books', 'picture-books']
            
def get_2015_2016_books():
    
    for year in years:
        
        print(year)
        
        for genre in genres:
            
            print(genre)
            
            url = f'https://www.goodreads.com/choiceawards/best-{genre}-{year}'
            resp = urllib.request.urlopen(url)
            soup = BS(resp, from_encoding=resp.info().get_param('charset'))
            
            for info in soup.find_all(href=True):
#                 print(info)

                if 'from_choice=true' in info['href']:
                    end_url = info['href']

                    each_book = {}

                    resp = urllib.request.urlopen(f'https://www.goodreads.com{end_url}')
                    content = BS(resp, 'html.parser')

                    each_book['title'] = get_title(content)
                    each_book['author'] = get_author(content)
                    each_book['rating'] = get_rating(content)
                    each_book['pages'] = get_pages(content)
                    each_book['publisher'] = get_publisher(content)
                    each_book['ISBN'] = get_ISBN(content)
                    each_book['genre'] = get_genre(content)
                    each_book['series'] = get_series(content)
                    each_book['author_followers'] = get_followers(content)
                    each_book['publish_date'] = get_release_date(content)

                    books_2015_2016.append(each_book)

                    print(f"Inserted {each_book['title']}")

    pd.DataFrame(books_2015_2016).to_csv('books_2018.csv')

    return books_2015_2016

In [231]:
get_2015_2016_books()

2015
fiction-books
Inserted Go Set a Watchman
Inserted Go Set a Watchman
Inserted Go Set a Watchman
Inserted After You
Inserted The Royal We
Inserted A Little Life
Inserted My Grandmother Asked Me to Tell You She's Sorry
Inserted Fates and Furies
Inserted Inside the O'Briens
Inserted Girl at War
Inserted Our Souls at Night
Inserted Trail of Broken Wings
Inserted The Sound of Glass
Inserted Kitchens of the Great Midwest
Inserted First Frost
Inserted The Secret Wisdom of the Earth
Inserted Did You Ever Have a Family
Inserted When the Moon is Low
Inserted A Spool of Blue Thread
Inserted The Perfect Son
Inserted Signs Preceding the End of the World
Inserted Purity
mystery-thriller-books
Inserted The Girl on the Train
Inserted The Girl on the Train
Inserted The Girl on the Train
Inserted Finders Keepers
Inserted Career of Evil
Inserted Obsession in Death
Inserted Pretty Girls
Inserted Marrow
Inserted The Nature of the Beast
Inserted The English Spy
Inserted Die Again
Inserted The Stranger
I

Inserted The Billion Dollar Spy: A True Story of Cold War Espionage and Betrayal
Inserted Ashley's War: The Untold Story of a Team of Women Soldiers on the Special Ops Battlefield
Inserted Rosemary: The Hidden Kennedy Daughter
Inserted The Dead Duke, His Secret Wife, and the Missing Corpse: An Extraordinary Edwardian Case of Deception and Intrigue
Inserted Killing Reagan
Inserted A Kim Jong-Il Production: The Extraordinary True Story of a Kidnapped Filmmaker, His Star Actress, and a Young Dictator's Rise to Power
Inserted Legend: The Incredible Story of Green Beret Sergeant Roy Benavidez's Heroic Mission to Rescue a Special Forces Team Caught Behind Enemy Lines
Inserted Lafayette in the Somewhat United States
Inserted The Residence: Inside the Private World of the White House
science-technology-books
Inserted Beneath the Surface: Killer Whales, SeaWorld, and the Truth Beyond Blackfish
Inserted Beneath the Surface: Killer Whales, SeaWorld, and the Truth Beyond Blackfish
Inserted Beneath

Inserted Fish in a Tree
Inserted The Penderwicks in Spring
Inserted Goodbye Stranger
Inserted Echo
Inserted Rogue Knight
Inserted The Marvels
Inserted The Honest Truth
Inserted The Copper Gauntlet
Inserted The Evertree
Inserted Crenshaw
Inserted The Hollow Boy
picture-books
Inserted The Day the Crayons Came Home
Inserted The Day the Crayons Came Home
Inserted The Day the Crayons Came Home
Inserted Winnie: The True Story of the Bear Who Inspired Winnie-the-Pooh
Inserted I Will Take a Nap!
Inserted What Pet Should I Get?
Inserted The Princess and the Pony
Inserted Goodnight Already!
Inserted If You Plant a Seed
Inserted Red: A Crayon's Story
Inserted I Wish You More
Inserted Sidewalk Flowers
Inserted Wolfie the Bunny
Inserted Mesmerized: How Ben Franklin Solved a Mystery that Baffled All of France
Inserted This Is Sadie
Inserted Last Stop on Market Street
Inserted Your Baby's First Word Will Be DADA
Inserted Interstellar Cinderella
Inserted A Fine Dessert: Four Centuries, Four Families, 

Inserted The Rainbow Comes and Goes: A Mother and Son On Life, Love, and Loss
Inserted Love Warrior
Inserted Born to Run
Inserted Lab Girl
Inserted Hillbilly Elegy: A Memoir of a Family and Culture in Crisis
Inserted Hungry Heart: Adventures in Life, Love, and Writing
Inserted Buffering: Unshared Tales of a Life Fully Loaded
Inserted Shrill: Notes from a Loud Woman
Inserted A Mother's Reckoning: Living in the Aftermath of Tragedy
Inserted A Thousand Naked Strangers: A Paramedic's Wild Ride to the Edge and Back
Inserted Unashamed
Inserted The Sound of Gravel: A Memoir
Inserted Party of One: A Memoir in 21 Songs
Inserted Lust & Wonder
Inserted Shoe Dog: A Memoir by the Creator of NIKE
Inserted Sex Object: A Memoir
Inserted Keeping Kyrie
Inserted A Smile in One Eye: a Tear in the Other
history-biography-books
Inserted Leonard: My Fifty-Year Friendship with a Remarkable Man
Inserted Leonard: My Fifty-Year Friendship with a Remarkable Man
Inserted Leonard: My Fifty-Year Friendship with a Re

Inserted If I Was Your Girl
Inserted The Serpent King
Inserted The Way I Used to Be
Inserted Symptoms of Being Human
Inserted Love & Gelato
Inserted Tell Me Three Things
Inserted Walk the Edge
Inserted When We Collided
Inserted The Female of the Species
Inserted First Kiss
Inserted Inferno
young-adult-fantasy-books
Inserted A Court of Mist and Fury
Inserted A Court of Mist and Fury
Inserted A Court of Mist and Fury
Inserted Lady Midnight
Inserted Empire of Storms
Inserted Crooked Kingdom
Inserted The Raven King
Inserted Glass Sword
Inserted The Crown
Inserted Calamity
Inserted Heartless
Inserted A Torch Against the Night
Inserted We Are the Ants
Inserted The Rose & the Dagger
Inserted The Winner's Kiss
Inserted United as One
Inserted Rebel of the Sands
Inserted Gemina
Inserted My Lady Jane
Inserted The Midnight Star
Inserted And I Darken
Inserted Ivory and Bone
childrens-books
Inserted The Hidden Oracle
Inserted The Hidden Oracle
Inserted The Hidden Oracle
Inserted Pax
Inserted The Ham

[{'title': 'Go Set a Watchman',
  'author': 'Harper Lee',
  'rating': 3.31,
  'pages': '278',
  'publisher': 'HarperCollins',
  'ISBN': '0062409859',
  'genre': 'Fiction',
  'series': 1,
  'author_followers': 10648,
  'publish_date': datetime.datetime(2015, 7, 14, 0, 0)},
 {'title': 'Go Set a Watchman',
  'author': 'Harper Lee',
  'rating': 3.31,
  'pages': '278',
  'publisher': 'HarperCollins',
  'ISBN': '0062409859',
  'genre': 'Fiction',
  'series': 1,
  'author_followers': 10648,
  'publish_date': datetime.datetime(2015, 7, 14, 0, 0)},
 {'title': 'Go Set a Watchman',
  'author': 'Harper Lee',
  'rating': 3.31,
  'pages': '278',
  'publisher': 'HarperCollins',
  'ISBN': '0062409859',
  'genre': 'Fiction',
  'series': 1,
  'author_followers': 10648,
  'publish_date': datetime.datetime(2015, 7, 14, 0, 0)},
 {'title': 'After You',
  'author': 'Jojo Moyes',
  'rating': 3.72,
  'pages': '352',
  'publisher': 'Pamela Dorman Books(first published September 23rd 2015)',
  'ISBN': '052542659